# The `Scalar object`

In `mlfz`, computational graphs are implemented by the `Scalar` class. (And also the `Tensor` class, but to flatten the learning curve, we'll stick to vanilla 
computational graphs without vectorization.)

You can instantiate a `Scalar` by either setting its value, or using operations and functions on other `Scalar`-s.

In [3]:
from mlfz.nn.scalar import Scalar

a = Scalar(3)
b = Scalar(-2)

a + b

Scalar(1)

As `Scalar` represents a node in a computational graph, each one holds
* a numeric value,
* 